In [6]:
import pandas as pd
from datasets import Dataset

# Đọc dữ liệu từ file CSV
df = pd.read_csv("./data/Dataset_articles_converted.csv")

# Chọn 30,000 dòng đầu tiên
df = df.head(10000)

# Chuyển đổi DataFrame sang Hugging Face Dataset
dataset = Dataset.from_pandas(df)

df.to_csv("Dataset_articles_10000.csv", index=False)
print("Dữ liệu đã được lưu thành Dataset_articles_10000.csv")


Dữ liệu đã được lưu thành Dataset_articles_10000.csv


In [9]:
from transformers import T5Tokenizer

model_name = "t5-small"  # hoặc tên model mà bạn muốn sử dụng
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    # Tạo input với prefix "summarize: "
    inputs = ["summarize: " + str(text) for text in examples["Contents"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=False)
    
    # Tokenize labels (tóm tắt)
    summaries = [str(summary) for summary in examples["Summary"]]
    labels = tokenizer(summaries, max_length=150, truncation=True, padding=False)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset đã cắt (không áp dụng padding cố định ở đây)
tokenized_datasets = dataset.map(preprocess_function, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
# Chia dataset thành tập train và eval (ví dụ 90% train, 10% eval)
split_dataset = tokenized_datasets.train_test_split(test_size=0.8, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Load tokenizer và mô hình (ví dụ sử dụng t5-small)
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Data collator để padding động
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
    save_total_limit=1,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


  0%|          | 0/1250 [00:00<?, ?it/s]

{'loss': 2.1629, 'grad_norm': 1.1291643381118774, 'learning_rate': 2.52e-05, 'epoch': 0.8}
{'loss': 1.9933, 'grad_norm': 1.208406686782837, 'learning_rate': 2.04e-05, 'epoch': 1.6}
{'loss': 1.9384, 'grad_norm': 1.3569822311401367, 'learning_rate': 1.56e-05, 'epoch': 2.4}
{'loss': 1.9122, 'grad_norm': 1.7546330690383911, 'learning_rate': 1.08e-05, 'epoch': 3.2}
{'loss': 1.8835, 'grad_norm': 1.1515312194824219, 'learning_rate': 6e-06, 'epoch': 4.0}


  0%|          | 0/4000 [00:00<?, ?it/s]

{'eval_loss': 1.8001431226730347, 'eval_runtime': 1337.0228, 'eval_samples_per_second': 5.983, 'eval_steps_per_second': 2.992, 'epoch': 4.0}
{'loss': 1.8722, 'grad_norm': 1.4307829141616821, 'learning_rate': 1.2000000000000002e-06, 'epoch': 4.8}
{'train_runtime': 7655.2713, 'train_samples_per_second': 1.306, 'train_steps_per_second': 0.163, 'train_loss': 1.9566175048828125, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=1.9566175048828125, metrics={'train_runtime': 7655.2713, 'train_samples_per_second': 1.306, 'train_steps_per_second': 0.163, 'total_flos': 1350823787298816.0, 'train_loss': 1.9566175048828125, 'epoch': 5.0})

model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")


In [12]:
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")


('./fine_tuned_t5\\tokenizer_config.json',
 './fine_tuned_t5\\special_tokens_map.json',
 './fine_tuned_t5\\spiece.model',
 './fine_tuned_t5\\added_tokens.json')

In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model đã fine-tune
model = T5ForConditionalGeneration.from_pretrained("./fine_tuned_t5")
tokenizer = T5Tokenizer.from_pretrained("./fine_tuned_t5")

# Test với một văn bản mới
def summarize(text):
    input_ids = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).input_ids
    output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

# Ví dụ tóm tắt
text = "Đây là một đoạn văn dài cần được tóm tắt bằng mô hình đã fine-tune."
print("Tóm tắt:", summarize(text))

Tóm tắt: ây là mt on văn dài cn c tóm tt bng mô hnh  fine-tune.


In [16]:
df.head()

,Contents,Summary
0,"Những ngày vừa qua, trên trang Facebook chính ...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn..."
1,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...",TPHCM - Việc không thể cưỡng chế thuế của hai ...
2,CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ..."
3,Hoàn công nhà ở là một thủ tục hành chính tron...,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...
4,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,Có rất nhiều lý do khiến những dự án thấp nội ...
